In [4]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma

model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}
embedding = HuggingFaceBgeEmbeddings(
                model_name=model_name,
                model_kwargs=model_kwargs,
                encode_kwargs=encode_kwargs,
                query_instruction="Index for text",
                # cache_folder="~/.cache/huggingface/hub"
            )

In [5]:
from langchain.document_loaders import TextLoader, DirectoryLoader
loader = TextLoader("conversation1.txt")
# loader = DirectoryLoader(f'MedQA/data_clean/textbooks/en', glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1000, chunk_overlap=200, add_start_index=True
# )
text_splitter = RecursiveCharacterTextSplitter(
    separators=[
        "\n\n",
    ],
    add_start_index=True,
    chunk_size=1000,
    # Existing args
)
documents = text_splitter.split_documents(documents)
documents[:5]

[Document(page_content='patient: I woke up this morning feeling the whole room is spinning when i was sitting down. I went to the bathroom walking unsteadily, as i tried to focus i feel nauseous. I try to vomit but it wont come out.. After taking panadol and sleep for few hours, i still feel the same.. By the way, if i lay down or sit down, my head do not spin, only when i want to move around then i feel the whole world is spinning.. And it is normal stomach discomfort at the same time? Earlier after i relieved myself, the spinning lessen so i am not sure whether its connected or coincidences.. Thank you doc!  doctor: Hi, Thank you for posting your query. The most likely cause for your symptoms is benign paroxysmal positional vertigo (BPPV), a type of peripheral vertigo. In this condition, the most common symptom is dizziness or giddiness, which is made worse with movements. Accompanying nausea and vomiting are common. The condition is due to problem in the ear, and improves in a few d

In [9]:
persist_directory = 'bge1'
db = Chroma.from_documents(documents, embedding, persist_directory=persist_directory)
# similarity search
db.similarity_search("what should i do if i have a fever")

db.persist()
db = None

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/mistgpu/site-packages/langchain_community/vectorstores/chroma.py", line 82, in __init__
    import chromadb
  File "/mistgpu/site-packages/chromadb/__init__.py", line 1, in <module>
    import chromadb.config
  File "/mistgpu/site-packages/chromadb/config.py", line 1, in <module>
    from pydantic import BaseSettings
  File "/mistgpu/site-packages/pydantic/__init__.py", line 386, in __getattr__
  File "/mistgpu/site-packages/pydantic/_migration.py", line 278, in wrapper
ModuleNotFoundError: No module named 'pydantic._internal'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3397, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_473/4105539457.py", line 2, in <cell line: 2>
    db = Chroma.from_documents(documents, embedding, persist_directory=persist_directory)


In [12]:
db = Chroma(persist_directory=persist_directory, embedding_function=embedding)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 20})


question = "i have a cough and headache"
retrieved_docs = retriever.invoke(question)

retrieved_docs

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[Document(page_content='patient: I had an upper resp. infection a few weeks ago which seems to have improved.  Now I have a deep cough (meaning that I have to keep coughing and pretty hard for the sensation to stop) that gets really bad when I lie down.  Now my head is beginning to hurt when I cough as well.  I dont have any more congestion or a runny nose.  I do still cough up gunk though.  doctor: Hello rearward welcome to Chat Doctor.come have evaluated your query thoroughly.* This is in relation with respiratory congestion from bacterial or viral infection.* Suggestions for better recovery.- Chat Doctor. - Gargles with salted lukewarm water and added peppermint oil 3 times a day.- Avoid exposure to pollen or dusty environment.- Use face mask liberally when needed.- Lesson pearls, tab. Tylenol (500) mg for symptomatic relief.- Avoid smoking in any form. Hope this will help you for sure. Welcome for further queries. Regards dear take care.', metadata={'source': 'conversation1.txt', '

In [13]:
loader = DirectoryLoader(f'MedQA/data_clean/textbooks/en', glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()

In [14]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)

In [15]:
documents = text_splitter.split_documents(documents)
documents[:5]

[Document(page_content='Growth Adaptations, Cellular Injury, and Cell Death\n\nI. BASIC PRINCIPLES\n\nA. An organ is in homeostasis with the physiologic stress placed on it.\n\nB. An increase, decrease, or change in stress on an organ can result in growth adaptations.\n\nII. HYPERPLASIA AND HYPERTROPHY\n\nA. An increase in stress leads to an increase in organ size.\n\n1. Occurs via an increase in the size (hypertrophy) and/or the number (hyperplasia) of cells\n\nB. Hypertrophy involves gene activation, protein synthesis, and production of organelles.\n\nC. Hyperplasia involves the production of new cells from stem cells.\n\nD. Hyperplasia and hypertrophy generally occur together (e.g., uterus during pregnancy).\n\n1.\n\nPermanent tissues (e.g., cardiac muscle, skeletal muscle, and nerve), however, cannot make new cells and undergo hypertrophy only.\n\n2.\n\nFor example, cardiac myocytes undergo hypertrophy, not hyperplasia, in response to systemic hypertension (Fig. 1.1).', metadata={'

In [16]:
db.add_documents(documents)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

['4f14f545-7c30-41e8-8ae3-78c04598e9de',
 '8a5d281f-1b2e-441e-8b67-3b461a8077a8',
 '69dfbae9-60b0-4163-843b-f3563f6dc208',
 '46cdc34d-6b47-4a84-b58f-3b6dfbd72386',
 '809af81d-d1d5-4c69-a422-d5d7620c1406',
 'cf343de7-8a83-4d8b-af28-68e780b8cfe3',
 'e2fb366c-bbc8-44fc-8fd3-a158abd7f4aa',
 '6c071dfa-7a4e-4774-a65c-74e339eeac75',
 'a76bae04-6177-4bf9-bb03-440126873fb2',
 '258e7ce0-0345-46d3-a3dc-d4301bf07036',
 '3fba9a3e-f262-4634-92e5-c292c99c33af',
 '1e4e1b12-0afb-49c6-869c-62345a09828e',
 '7e745f18-9559-4afc-9611-cf8e04fd97ac',
 '3f449e1e-85ae-4465-933c-cb8e5388e992',
 'fd19a4cb-8995-400d-b9d9-b43c20f88b41',
 '7913dd42-3a63-4f55-aeb6-bae579e1bae4',
 '01138b9d-2838-4dfc-a7f4-98f3c7456d27',
 '428214f4-b909-42a7-b278-3f93be37baf3',
 'a701fcf0-1fab-4f2c-822a-73187fcc0a4c',
 '3bbbf7bc-1753-4b5d-a942-7db15e195064',
 '7a1f7065-16bc-4e15-8f92-16a8c4b7234d',
 '42c9ca26-545a-45c0-920c-05dfbc154d1f',
 'a7b9d2e8-d4a2-4a77-9372-e23034a56ce5',
 '6040c728-2c3e-4b31-827b-0ca7a9fa18c1',
 '95f748b9-8040-

In [17]:
db.persist()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [20]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 20})


question = "Cardiovascular system: Delivers the fuels to the cells and to and from their storage sites"
retrieved_docs = retriever.invoke(question)

retrieved_docs

[Document(page_content='KEY A, atrium A, small artery AT, adipose tissue AVN, atrioventricular node B, blood CA, coronary artery CB, conduction bundle CM, cardiac muscle CT, connective tissue CV, cardiac vein DCT, dense connective tissue En, endothelium ID, intercalated disc IEM, internal elastic membrane LN, lymph node MV, mitral valve PF, Purkinje fibers SM, smooth muscle TA, tunica adventitia TM, tunica media TI, tunica intima V, ventricle', metadata={'source': 'MedQA/data_clean/textbooks/en/Histology_Ross.txt', 'start_index': 1447243}),
 Document(page_content='Fig. 13.1\n\nB), which have the enzymes necessary for oxidative phosphorylation, allows rapid oxidation of substrates and synthesis of ATP and thus sustains the myocardial energy requirements.\n\nCHAPTER 16 Elements of Cardiac Function\n\nTo provide adequate O2 and substrate for its metabolic machinery, the myocardium is also endowed with a rich capillary supply, approximately one capillary per fiber. Thus diffusion distances